In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [3]:
df = pd.read_csv("edu_enrollees.csv")
df.head(2)

,enrollee_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,Xgrp
0,8949.0,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36.0,1.0,train
1,29725.0,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47.0,0.0,train


### 전처리

In [4]:
df = df.drop(columns = ["city", "company_size", "company_type"])

In [7]:
# df.dtypes
df.isna().sum()

In [9]:
df = df.dropna()
df.isna().sum()

In [ ]:
df["experience"].unique(), df["last_new_job"].unique()

In [11]:
df = df.loc[~df["experience"].isin([">20", "<1"]), ]
df = df.loc[~df["last_new_job"].isin([">4", "never"]), ]

In [13]:
df["experience"] = df["experience"].astype("int")
df["last_new_job"] = df["last_new_job"].astype("int")

In [12]:
len(df)

7522

In [16]:
df_base = df.reset_index(drop = True)

### Q1.

In [17]:
df_q1 = df_base[["relevant_experience", "target"]].copy()
df_q1.head(2)

,relevant_experience,target
0,Has relevant experience,1.0
1,Has relevant experience,0.0


In [18]:
df_q1["relevant_experience"].unique()

array(['Has relevant experience', 'No relevant experience'], dtype=object)

In [19]:
# 없는 -> A, 있는 -> B
df_A = df_q1.loc[df_q1["relevant_experience"] ==  "No relevant experience", ]
df_B = df_q1.loc[df_q1["relevant_experience"] == "Has relevant experience", ]

In [22]:
df_A["target"].value_counts(normalize = True)

target
0.0    0.617127
1.0    0.382873
Name: proportion, dtype: float64

In [23]:
df_B["target"].value_counts(normalize = True)

target
0.0    0.784089
1.0    0.215911
Name: proportion, dtype: float64

In [24]:
stat_A = df_A["target"].value_counts(normalize = True)[1]
stat_B = df_B["target"].value_counts(normalize = True)[1]
round(stat_A / stat_B, 2)

1.77

In [25]:
df_q1.groupby("relevant_experience")["target"].mean()

relevant_experience
Has relevant experience    0.215911
No relevant experience     0.382873
Name: target, dtype: float64

### Q2.

In [28]:
df_q2_obj = df_base.iloc[:, 2:7].copy()
df_q2_obj.head(2)

,gender,relevant_experience,enrolled_university,education_level,major_discipline
0,Male,Has relevant experience,no_enrollment,Graduate,STEM
1,Male,Has relevant experience,no_enrollment,Graduate,STEM


In [31]:
# df_q2_dum = pd.get_dummies(df_q2_obj, columns = df_q2_obj.columns) # 시험버전
df_q2_dum = pd.get_dummies(df_q2_obj, dtype = "int") # Pandas 2.0.0 이상

In [32]:
df_q2_dum.head(2)

,gender_Female,gender_Male,gender_Other,relevant_experience_Has relevant experience,relevant_experience_No relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,education_level_Masters,education_level_Phd,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM
0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1
1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1


In [35]:
df_q2_obj.columns[[2, 3]]

Index(['enrolled_university', 'education_level'], dtype='object')

In [ ]:
df_q2_dum.columns.to_series().reset_index()

In [38]:
df_q2_dum = df_q2_dum.drop(columns = df_q2_dum.columns[[2, 4, 7, 10, 16]])

In [39]:
df_q2_dum.head(1)

,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,0,1,1,0,0,1,0,0,0,0,0,0


In [75]:
df_base.columns[np.r_[10, 11, 1:5]]

Index(['target', 'Xgrp', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university'],
      dtype='object')

In [40]:
cols = ["city_development_index", "experience", "last_new_job", "training_hours"]
df_job2 = pd.concat([df_base[["target", "Xgrp"] + cols],
                     df_q2_dum],
                    axis = 1)
df_job2.head(2)

,target,Xgrp,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,1.0,train,0.92,7,1,46.0,0,1,1,0,0,1,0,0,0,0,0,0
1,0.0,train,0.92,5,1,108.0,0,1,1,0,0,1,0,0,0,0,0,0


In [ ]:
model_lr = LogisticRegression(C = 100000, max_iter = 1000,
                              solver = "liblinear", random_state = 123)
model_lr.fit(X = df_job2.drop(columns = ["target", "Xgrp"]),
             y = df_job2["target"])

In [45]:
model_lr.coef_ # 회귀 계수

array([[-6.11732384e+00, -2.85015530e-02,  9.56531984e-02,
        -9.26206002e-04, -1.62975865e-01, -1.36723955e-01,
        -7.60567385e-01,  5.14109835e-01, -2.81496522e-01,
         3.23116412e-01,  1.67418791e-02,  2.61665001e-01,
         1.07607195e-01,  2.43479152e-01,  3.97900227e-01,
        -4.39387752e-01]])

In [52]:
pd.DataFrame(model_lr.coef_ , columns = df_job2.columns[2:]).transpose()

In [ ]:
pd.DataFrame(model_lr.coef_ , 
             columns = df_job2.columns[2:]).transpose()

In [53]:
pd.DataFrame(np.exp(model_lr.coef_), # OR, Odds Ratio
             columns = df_job2.columns[2:]).transpose()

,0
city_development_index,0.002204
experience,0.971901
last_new_job,1.100377
training_hours,0.999074
gender_Female,0.849612
gender_Male,0.872211
relevant_experience_Has relevant experience,0.467401
enrolled_university_Full time course,1.672149
enrolled_university_Part time course,0.754654
education_level_Graduate,1.381426


In [55]:
np.exp(model_lr.coef_).max()

1.6721493496611732

In [56]:
np.floor(1.2345)

1.0

### Q3.

In [57]:
df_train = df_job2.loc[df_job2["Xgrp"] == "train", ]
df_test  = df_job2.loc[df_job2["Xgrp"] == "test",  ]
len(df_train), len(df_test)

(4706, 2816)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors = 5)
model_knn.fit(X = df_train.drop(columns = ["target", "Xgrp"]),
              y = df_train["target"])
pred = model_knn.predict(df_test.drop(columns = ["target", "Xgrp"]))
pred[:4]

In [60]:
y_t = df_test["target"]
y_p = pred

In [62]:
df_tab = pd.crosstab(y_t, y_p)
df_tab

col_0,0.0,1.0
target,,
0.0,1899,193
1.0,616,108


In [70]:
df_tab.iloc[0, 1]

In [71]:
df_tab.values.diagonal().sum() / df_tab.values.sum()

In [68]:
from sklearn.metrics import accuracy_score

In [69]:
round(accuracy_score(y_true = y_t, y_pred = y_p), 2)

0.71

여러분의 시험과 가장 비슷한 시험은 멀티캠퍼스의 ProDS 자격의 Advanced level 입니다.
그러나....  
아쉽게도 관련 서적 0, 비슷한 예제 0

그나마 빅데이터 분석기사....로 검색했을 때 뭔가가 나오긴 함....